# Introduction
This notebook contains a basic breakout strategy that can be used as a starting point to build and test your own strategy upon.

In [ ]:
%use roboquant(1.3.0-SNAPSHOT)
Welcome()

In [ ]:
/**
 * Basic breakout strategy.
 *
 * @param period the historic period to use for checking this is a breakout
 */
class MyStrategy(period: Int) : HistoricPriceStrategy(period) {

    /**
     * This method will be called by the superclass for every asset in the feed once
     * there is enough data collected for that asset.
     */
    override fun generateRating(data: DoubleArray): Rating? {
        return when (data.last()) {
            data.max() -> Rating.BUY
            data.min() -> Rating.SELL
            else -> null
        }
    }

}

In [ ]:
// Create an instance of MyStrategy with 26 days worth of history
val strategy = MyStrategy(period = 26)

// Define the metric(s) you want to monitor
val metric = AccountMetric()

// Create an instance of Roboquant
val roboquant = Roboquant(strategy, metric)

// Create the data feed that will be used during the back test
val feed = AvroFeed.sp500()

In [ ]:
// Run the back test
roboquant.run(feed)

In [ ]:
// Print the account summary
roboquant.broker.account.summary()

In [ ]:
// Plot the equity metric
val equity = roboquant.logger.getMetric("account.equity")
MetricChart(equity)